In [50]:
import streamlit as st
import pandas as pd
import numpy as np
import pytz

import psycopg2
import plotly.express as px
import datetime

def init_connection():
    return psycopg2.connect(**st.secrets["postgres"])
conn = init_connection()

In [53]:
def load_data(nrows):
    df = pd.read_sql_query('select * from "atendimentos"',con=conn)
    return df

data = load_data(1000)
data

,id,tipo_senha,numeracao,codigo_senha,data_emissao,guiche,data_atendimento
0,24,SP,7,SP007,2022-09-06 22:41:47.444103,1.0,2022-09-06 23:07:19.874273
1,28,SE,2,SE002,2022-09-06 22:55:15.804991,3.0,2022-09-06 23:07:23.199304
2,25,SP,8,SP008,2022-09-06 22:43:22.651726,1.0,2022-09-06 23:07:26.636278
3,29,SE,3,SE003,2022-09-06 22:55:17.389104,1.0,2022-09-06 23:07:29.997432
4,26,SP,9,SP009,2022-09-06 22:44:17.792339,2.0,2022-09-06 23:07:33.401444
5,30,SE,4,SE004,2022-09-06 22:55:19.358321,2.0,2022-09-06 23:07:36.847689
6,32,SP,5,SP005,2022-09-06 22:55:34.518070,5.0,2022-09-06 23:07:40.222096
7,31,SE,5,SE005,2022-09-06 22:55:20.607535,5.0,2022-09-06 23:07:43.652054
8,33,SP,1,SP001,2022-09-07 00:43:47.199765,NaN,NaT
9,34,SG,1,SG001,2022-09-07 00:53:21.850429,NaN,NaT


In [60]:
from dateutil.relativedelta import relativedelta




['SP', 'SE', 'SG']

In [30]:
data.data_emissao[0].month

9

In [27]:
data['hora_emissao'] = data['data_emissao'].dt.hour
data['hora_atendimento'] = data['data_atendimento'].dt.hour

In [29]:
data.rename(columns = {"hora_emissao":"Hora Emissao", "id":"Count"}).groupby(by=["Hora Emissao"]).count()[["Count"]].sort_values(by="Count")

,Count
Hora Emissao,
0,1
1,7
22,11


In [25]:
data.groupby(by=["tipo_senha"]).count()[["id"]].sort_values(by="id")

,id
tipo_senha,
SE,5
SP,14


In [24]:
data.groupby('tipo_senha').size()

tipo_senha
SE     5
SP    14
dtype: int64

In [13]:
tipos = data['tipo_senha'].unique()
for tipo in tipos:
    cond1 = ~data["data_atendimento"].isna()
    cond2 = data["tipo_senha"] == tipo
    senha_por_tipo = int(len(data[cond1 & cond2]))
    print(f"Senhas Atendidas {tipo}: {senha_por_tipo}")

Senhas Atendidas SE: 5
Senhas Atendidas SP: 13
